In [ ]:
!pip install konlpy

In [ ]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
from konlpy.tag import Hannanum  # 한나눔으로 변경

!apt -qq -y install fonts-nanum > /dev/null

# 형태소 분석기 초기화 (한나눔으로 변경)
hannanum = Hannanum()

# 데이터 불러오기
train_data = pd.read_csv('/content/코퍼스데이터.csv')
print('데이터 개수:', len(train_data))

# 문자열이 아닌 데이터 제거
train_review = train_data['댓글'].dropna().map(str)

# 불용어 목록 파일을 불러옵니다.
with open('/content/drive/MyDrive/stopword_main.txt', 'r', encoding='utf-8') as f:
    stopwords = np.array([line.strip().replace('\r', '') for line in f.readlines()])

# 명사 추출 및 불용어 처리 (한나눔 사용)
filtered_review = []
for review in tqdm(train_review, desc="리뷰 처리 중"):
    review = review.lower()
    review = re.sub(r'[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-zA-Z0-9]+', '', review)
    nouns = hannanum.nouns(review)  # 한나눔 형태소 분석을 통해 명사만 추출

    # numpy 배열을 사용하여 불용어를 필터링합니다. 이 경우, 리스트 컴프리헨션을 사용하는 것이 더 적합합니다.
    filtered_nouns = ' '.join([noun for noun in nouns if noun not in stopwords])
    filtered_review.append(filtered_nouns)

# 다시 numpy 배열로 변환합니다.
filtered_review = np.array(filtered_review)

# 워드 클라우드 생성 및 출력
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
wordcloud = WordCloud(font_path=fontpath, background_color='white', width=800, height=800, stopwords=set(stopwords)).generate(' '.join(filtered_review))

plt.figure(figsize=(8, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
